In [ ]:
import sys
import os
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, project_root)
#импорты загрузки 

import InterfaceWidgets.data_load_widgets as data_load_widgets
import InterfaceWidgets.column_selection_widgets as column_selection_widgets

#импорты EDA
import InterfaceWidgets.EDA_widgets as eda_widgets

#импорты экспериментального ядра
import ExperimentsCore.experiments_core as exp_core
from ExperimentsCore.experiments_engine import run_experiment
import InterfaceWidgets.experiment_config_widgets as exp_config_widgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Импорт визуализации
import DataVisualizations.visualizations as viz
from DataVisualizations.means_vis_bootstrap import visualize_means_distribution

# Создаем ГЛОБАЛЬНЫЙ контейнер для приложения - это основное изменение
main_container = widgets.VBox(layout=widgets.Layout(width='100%'))
display(main_container)

# Создаем секционные контейнеры для разных частей приложения
data_load_section = widgets.VBox()
column_selection_section = widgets.VBox()
column_info_section = widgets.VBox()
eda_section = widgets.VBox()
distribution_section = widgets.VBox()
experiment_section = widgets.VBox()
# Новая секция для распределения средних
means_distribution_section = widgets.VBox()

# Добавляем первую секцию (загрузка данных) в главный контейнер
main_container.children = [data_load_section]

# Переменные для хранения текущих объектов
current_statistics = None
current_rv = None
current_df_dist = None

# Функция для расчета статистик распределения
def calculate_distribution_statistics(rv, df_dist):
    """
    Рассчитывает различные статистики для распределения.
    
    Args:
        rv: Объект дискретного распределения
        df_dist: DataFrame с значениями и вероятностями
        
    Returns:
        dict: Словарь с рассчитанными статистиками
    """
    stats = {}
    
    # Рассчитываем среднее
    if hasattr(rv, 'mean'):
        stats['mean'] = rv.mean()
    else:
        # Если у rv нет метода mean(), вычисляем вручную
        xk = df_dist['value'].values
        pk = df_dist['probability'].values
        stats['mean'] = np.sum(xk * pk)
    
    # Рассчитываем медиану (как квантиль 0.5)
    # Для дискретного распределения найдем значение, где CDF >= 0.5
    try:
        median_idx = np.where(df_dist['cumulative_probability'] >= 0.5)[0][0]
        stats['median'] = df_dist['value'].iloc[median_idx]
    except (IndexError, KeyError):
        stats['median'] = None
    
    # В будущем здесь можно добавить расчет других статистик
    # stats['quantile'] = ...
    # stats['proportion'] = ...
    
    return stats

# Функция для создания сокращенного представления DataFrame
def create_abbreviated_df(df, max_rows=10):
    """
    Создает сокращенное представление DataFrame, показывая первые и последние строки.
    
    Args:
        df: DataFrame для сокращения
        max_rows: Максимальное число строк для полного отображения
        
    Returns:
        DataFrame: Сокращенное представление
    """
    if len(df) <= max_rows:
        return df
    else:
        # Берем первые 5 и последние 5 строк
        head_df = df.head(5)
        tail_df = df.tail(5)
        
        # Добавляем разделитель между первыми и последними строками
        separator_df = pd.DataFrame([{col: '...' for col in df.columns}])
        
        # Объединяем в единый DataFrame для отображения
        return pd.concat([head_df, separator_df, tail_df])

# Обработчик обновления прогресса эксперимента
def update_experiment_progress(current_size, power, target_power, iteration, total_iterations):
    # Создаем виджет для отображения прогресса
    progress_info = widgets.HTML(
        f"Текущий размер выборки: {current_size}<br>"
        f"Текущая мощность: {power:.3f}<br>"
        f"Целевая мощность: {target_power}<br>"
        f"Прогресс: {iteration}/{total_iterations}"
    )
    
    # Обновляем секцию эксперимента
    experiment_section.children = [progress_info]

# Новая функция для отображения распределения средних
def display_means_distribution(rv, sample_size, mde_percent, statistic):
    """
    Отображает распределение средних в отдельной секции.
    
    Args:
        rv: Объект дискретного распределения
        sample_size: Оптимальный размер выборки
        mde_percent: Процент MDE
        statistic: Тип статистики
    """
    # Очищаем секцию распределения средних
    means_distribution_section.children = []
    
    # Создаем заголовок и информационное сообщение
    header = widgets.HTML("<h3>Распределение средних</h3>")
    loading_message = widgets.HTML(
        f"<h4 style='margin-top: 10px; color: #4a6baf;'>Генерация распределения средних для выборки размером {sample_size}...</h4>"
    )
    
    # Добавляем заголовок и сообщение в секцию
    means_distribution_section.children = [header, loading_message]
    
    # Добавляем секцию распределения средних в главный контейнер, если её там нет
    if means_distribution_section not in main_container.children:
        main_container.children = list(main_container.children) + [means_distribution_section]
    
    # Визуализируем распределение средних
    means_fig = visualize_means_distribution(
        rv, 
        sample_size, 
        mde_percent,
        statistic=statistic
    )
    
    # Преобразуем в FigureWidget для отображения
    means_fig_widget = go.FigureWidget(data=means_fig.data, layout=means_fig.layout)
    
    # Обновляем секцию распределения средних, заменяя информационное сообщение на график
    means_distribution_section.children = [header, means_fig_widget]

# Обработчик запуска эксперимента на основе конфигурации
def handle_experiment_config(config):
    global current_rv
    
    if current_rv is None:
        error_widget = widgets.HTML("<span style='color:red'>Ошибка: Не найдено дискретное распределение для эксперимента.</span>")
        experiment_section.children = [error_widget]
        return
    
    # Очищаем секцию эксперимента
    experiment_section.children = []
    
    # Создаем информационные виджеты
    config_info = widgets.HTML(f"<h3>Запуск эксперимента с конфигурацией:</h3>{str(config)}")
    
    # Создаем прогресс-бар
    progress_bar = widgets.FloatProgress(
        value=0,
        min=0,
        max=1.0,
        description='Прогресс:',
        bar_style='info',
        style={'bar_color': '#2C5E99'},
        orientation='horizontal'
    )
    
    # Добавляем виджеты в секцию эксперимента
    experiment_section.children = [config_info, progress_bar]
    
    # Запускаем эксперимент
    results = run_experiment(current_rv, config, on_progress_update=update_experiment_progress)
    
    # Подготавливаем виджеты для результатов
    result_widgets = []
    
    # Создаем график с помощью нашей функции визуализации
    fig = viz.power_analysis_plot(results, target_power=config['target_power'])
    
    # Преобразуем в FigureWidget для отображения в виджете
    fig_data = fig.data
    fig_layout = fig.layout
    figure_widget = go.FigureWidget(data=fig_data, layout=fig_layout)
    result_widgets.append(figure_widget)
    
    # Создаем сокращенную версию результатов
    abbreviated_results = create_abbreviated_df(results)
    
    # Создаем виджет для таблицы с результатами
    result_table = widgets.HTML(abbreviated_results.to_html())
    result_widgets.append(result_table)
    
    # Обновляем секцию эксперимента новыми виджетами
    experiment_section.children = result_widgets
    
    # Добавляем секцию эксперимента в главный контейнер, если её там нет
    if experiment_section not in main_container.children:
        main_container.children = list(main_container.children) + [experiment_section]
    
    # Если мы достигли целевой мощности, запускаем отображение распределения средних в отдельной секции
    if len(results) > 0 and results['power'].max() >= config['target_power']:
        # Находим минимальный размер выборки с достаточной мощностью
        optimal_sample_size = results.loc[results['power'] >= config['target_power'], 'sample_size'].min()
        
        # Запускаем отображение распределения средних в отдельной секции
        display_means_distribution(
            current_rv, 
            optimal_sample_size, 
            config['mde_percent'],
            config['statistic']
        )

# Обработчик создания распределения после EDA
def handle_distribution_creation(series):
    global current_statistics, current_rv, current_df_dist
    
    # Очищаем секцию распределения
    distribution_section.children = []
    
    # Создаем заголовок
    header = widgets.HTML("<h3>Создание дискретного распределения</h3>")
    
    # Создаем распределение
    current_rv, current_df_dist, round_digits = exp_core.discrete_dist_creation(series)
    
    # Сохраняем статистики для использования в интерфейсе конфигурации
    current_statistics = calculate_distribution_statistics(current_rv, current_df_dist)
    
    # Создаем сокращенное представление таблицы распределения
    display_df = create_abbreviated_df(current_df_dist)
    
    # Создаем виджеты для отображения информации о распределении
    df_html = widgets.HTML(display_df.to_html())
    
    # Добавляем виджеты в секцию распределения
    distribution_section.children = [header, df_html]
    
    # Добавляем секцию распределения в главный контейнер, если её там нет
    if distribution_section not in main_container.children:
        main_container.children = list(main_container.children) + [distribution_section]
    
    # Отображаем интерфейс конфигурации с передачей статистик распределения
    exp_config_widgets.display_experiment_config_interface(
        parent_container=main_container,
        current_statistics=current_statistics,
        on_config_created=handle_experiment_config
    )

# Обработчик выбора колонки
def handle_column_selection(selected_column, data_dict):
    # Очищаем секцию информации о колонке
    column_info_section.children = []
    
    # Создаем заголовок
    header = widgets.HTML(f"<h3>Выбрана колонка: {selected_column}</h3>")
    
    # Добавляем заголовок в секцию информации о колонке
    column_info_section.children = [header]
    
    # Добавляем секцию информации о колонке в главный контейнер, если её там нет
    if column_info_section not in main_container.children:
        main_container.children = list(main_container.children) + [column_info_section]
    
    # Запускаем EDA анализ для выбранной колонки
    series = data_dict['df'][selected_column]
    eda_widgets.run_eda_analysis(
        selected_column, 
        data_dict,
        parent_container=main_container,
        on_eda_complete=lambda: handle_distribution_creation(series)
    )

# Обработчик загрузки данных
def handle_data_loaded(data_dict):
    # Создаем селектор колонок и добавляем его в соответствующую секцию
    column_selection_widgets.column_selection_widgets(
        data_dict, 
        parent_container=column_selection_section,
        on_column_selected=handle_column_selection
    )
    
    # Добавляем секцию выбора колонки в главный контейнер, если её там нет
    if column_selection_section not in main_container.children:
        main_container.children = list(main_container.children) + [column_selection_section]

# Показываем интерфейс загрузки с callback'ом
data_load_widgets.display_interface(
    parent_container=data_load_section,
    on_data_loaded=handle_data_loaded)